In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
from tqdm import tqdm
import sys

sys.setrecursionlimit(5000)

# Gobblet Gobblers 환경 정의

In [2]:
class Environment:
    def __init__(self):
        # end_check 함수를 간단하게 해줄 게임 판
        self.board_v = np.zeros(9)
        # 밑에 있는 보드는 0으로 초기화된 27개의 배열로 준비
        self.board_r = np.zeros(27)

        # 게임 종료: done = True
        self.done = False
        self.reward = 0
        self.winner = 0
        self.print = True

        self.p1_small_piece = 2
        self.p1_medium_piece = 2
        self.p1_large_piece = 2

        self.p2_small_piece = 2
        self.p2_medium_piece = 2
        self.p2_large_piece = 2

    def move(self, player1, player2, player):
        # 각 플레이어가 선택한 행동을 표시하고 게임 상태(진행 또는 종료)를 판단
        # p1 = 1. p2 = -1로 정의
        # 긱 플레이어는 행동을 선택하는 select_action 메서드를 가짐

        if player == 1:
            pos = player1.select_action(self, player)

        else:
            pos = player2.select_action(self, player)

        # 보드에 플레이어의 선택을 표시(만약 새로운 말을 놓는 거라면)
        if isinstance(pos, int):  # 숫자라면
            self.board_r[pos] = player
        else:  # 문자라면 -> 움직이는 액션이다
            pos = pos.split('to')  # pos[0]가 지울 장소, pos[1]이 생길 장소

            # 원래 있던 자리에 있는 말을 지우고, (real 에서만 지우면 된다) 옮길 자리에 추가한다(마찬가지로 real 에서만 추가하면 된다.)
            self.board_r[int(pos[0])] = 0
            self.board_r[int(pos[1])] = player

        if self.print:
            self.print_board()

        # 보이는 보드로의 복사(end_check 함수를 간단히 만들기 위함)
        self.copy_real_to_vision()

        # 게임이 종료 상태인지 아닌지를 판단
        # print(player)
        self.end_check(player)
        return self.reward, self.done

    # 현재 보드 상태에서 가능한 행동(들 수 있는 장소)을 탐색하고 리스트로 반환
    def get_action(self, player):
        observation = []

        if player == 1:
            if self.p1_large_piece != 0:
                for i in range(18, 27):
                    if self.board_r[i] == 0:
                        observation.append(i)
            if self.p1_medium_piece != 0:
                for i in range(9, 18):
                    if self.board_r[i] == 0:
                        observation.append(i)
            if self.p1_small_piece != 0:
                for i in range(9):
                    if self.board_r[i] == 0:
                        observation.append(i)
        else:  # player == 2
            if self.p2_large_piece != 0:
                for i in range(18, 27):
                    if self.board_r[i] == 0:
                        observation.append(i)
            if self.p2_medium_piece != 0:
                for i in range(9, 18):
                    if self.board_r[i] == 0:
                        observation.append(i)
            if self.p2_small_piece != 0:
                for i in range(9):
                    if self.board_r[i] == 0:
                        observation.append(i)

        #  중간말이 놓인 위치에는 작은 말을 놓을 수 없다.(제거 작업)
        for i in range(9, 18):
            if self.board_r[i] != 0 and self.board_r[i - 9] == 0:
                if i-9 in observation:
                    observation.remove(i - 9)

        for i in range(18, 27):
            # 큰 말이 놓인 위치에는 중간말과 작은 말을 놓을 수 없다.
            if self.board_r[i] != 0 and self.board_r[i - 9] == 0:
                if i - 9 in observation:
                    observation.remove(i - 9)
            if self.board_r[i] != 0 and self.board_r[i - 18] == 0:
                if i - 18 in observation:
                    observation.remove(i - 18)

        observation.sort()  # 정렬
        
        # 이동 가능한 경우의 수 추가
        for i in range(9):
            if self.board_r[i] == player and self.board_r[i+9] == 0 and self.board_r[i+18] == 0:
                for j in range(9):
                    if self.board_r[j] == 0 and self.board_r[j+9] == 0 and self.board_r[j+18] == 0:
                        observation.append(str(i) + 'to' + str(j))

        for i in range(9, 18):
            if self.board_r[i] == player and self.board_r[i+9] == 0:
                for j in range(9,18):
                    if self.board_r[j] == 0 and self.board_r[j+9] == 0:
                        observation.append(str(i)+'to'+str(j))

        for i in range(18, 27):
            if self.board_r[i] == player:
                # 옮길 수 있는 위치 탐색
                for j in range(18, 27):
                    # 빈 공간이면
                    if self.board_r[j] == 0:
                        observation.append(str(i)+'to'+str(j))

        return observation

    # 게임이 종료(승패 또는 비김)됐는지 판단
    def end_check(self, player):
        # 0 1 2
        # 3 4 5
        # 6 7 8
        # 승패 조건은 가로, 세로, 대각선이 -1이나 1로 동일할 때
        end_condition = ((0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6))  # 승패 조건 생성
        
        # 이긴사람의 수 카운트 -> 두명 다 라인을 완성할 경우 비기므로
        p1_cnt = 0
        p2_cnt = 0
        # 승리 판별
        for line in end_condition:
            if self.board_v[line[0]] == self.board_v[line[1]] and \
                self.board_v[line[1]] == self.board_v[line[2]] and \
                    self.board_v[line[0]] == 1:  # 플레이어1 이 이겼다면
                # 종료됐다면 누가 이겼는지 표시
                self.done = True
                self.reward = 1
                p1_cnt += 1
            if self.board_v[line[0]] == self.board_v[line[1]] and \
                self.board_v[line[1]] == self.board_v[line[2]] and \
                    self.board_v[line[0]] == -1:  # 플레이어2 이 이겼다면
                # 종료됐다면 누가 이겼는지 표시
                self.done = True
                self.reward = -1
                p2_cnt += 1

        # 비긴 상태. 양쪽 모두 승리 조건을 동시에 만족하는 경우.
        if p1_cnt >= 1 and p2_cnt >= 1:
            self.done = True
            self.reward = 0
        return

    # 현재 보드의 상태를 표시: p1 = 0'', p2 = X''
    def print_board(self):
        print("+----+----+----+")
        for i in range(3):
            for j in range(3):
                if self.board_r[3 * i + j + 18] == 1:
                    print("| OL", end=" ")
                elif self.board_r[3 * i + j + 18] == -1:
                    print("| XL", end=" ")
                elif self.board_r[3 * i + j + 9] == 1:
                    print("| Om", end=" ")
                elif self.board_r[3 * i + j + 9] == -1:
                    print("| Xm", end=" ")
                elif self.board_r[3 * i + j] == 1:
                    print("| Os", end=" ")
                elif self.board_r[3 * i + j] == -1:
                    print("| Xs", end=" ")
                else:
                    print("|   ", end=" ")
            print("|")
            print("+----+----+----+")
        return
    
    # 맨 위의 게임 말만 표시하는 보드로 나타내기 end_check함수를 간단하게 작성하기 위함
    def copy_real_to_vision(self):
        self.board_v = np.zeros(9)
        for i in range(3):
            for j in range(3):
                if self.board_r[3 * i + j + 18] == 1:
                    self.board_v[3 * i + j] = 1
                elif self.board_r[3 * i + j + 18] == -1:
                    self.board_v[3 * i + j] = -1
                elif self.board_r[3 * i + j + 9] == 1:
                    self.board_v[3 * i + j] = 1
                elif self.board_r[3 * i + j + 9] == -1:
                    self.board_v[3 * i + j] = -1
                elif self.board_r[3 * i + j] == 1:
                    self.board_v[3 * i + j] = 1
                elif self.board_r[3 * i + j] == -1:
                    self.board_v[3 * i + j] = -1
        return

# 인간 플레이어

In [3]:
class Human_player:
    def __init__(self):
        self.name = "Human player"
        self.flag = 0

    def select_action(self, environment, player):
        while True:
            # 가능한 행동을 조사한 후 표시
            available_action = environment.get_action(player)

            print("possible actions = {}".format(available_action))

            # 상태 번호 표시
            print("+----+----+----+")
            print("+  0 +  1 +  2 +")
            print("+----+----+----+")
            print("+  3 +  4 +  5 +")
            print("+----+----+----+")
            print("+  6 +  7 +  8 +")
            print("+----+----+----+")

            # 키보드로 가능한 행동을 입력받음
            first_input = input("0-26 to move, n to place a new piece?")
            action = -1  # 선택한 행동이 선택할 수 없는 행동일 경우를 대비
            size = -1  # 선택한 행동이 size를 상관 안하면
            # 새로운 말을 놓겠다면
            if first_input == 'n':
                # 만약 새롭게 놓을 말이 없으면
                if player == 1 and environment.p1_small_piece + environment.p1_medium_piece + environment.p1_large_piece == 0:
                    # 놓을 수 없다!
                    self.flag = 1
                elif player == -1 and environment.p2_small_piece + environment.p2_medium_piece + environment.p2_large_piece == 0:
                    # 놓을 수 없다!
                    self.flag = 1

                else:
                    # 놓을 수 있는 말이 있으면 size를 입력 받는다
                    size = int(input("What size piece would you like to use? (0-2)"))
                    # 만약 그 사이즈에 해당하는 말이 없으면 놓을 수 없다.
                    if player == 1:
                        if size == 0 and environment.p1_small_piece == 0:
                            self.flag = 1
                        elif size == 1 and environment.p1_medium_piece == 0:
                            self.flag = 1
                        elif size == 2 and environment.p1_large_piece == 0:
                            self.flag = 1
                        else:
                            # 위치 입력받음
                            location = int(input("What square would you like to move to? (0-8)"))
                            action = location + size * 9

                    else:  # player == 2:
                        if size == 0 and environment.p2_small_piece == 0:
                            self.flag = 1
                        elif size == 1 and environment.p2_medium_piece == 0:
                            self.flag = 1
                        elif size == 2 and environment.p2_large_piece == 0:
                            self.flag = 1
                        else:
                            # 위치 입력받음
                            location = int(input("What square would you like to move to? (0-8)"))
                            action = location + size * 9

            elif first_input.isdigit():  # 옮기는 행동을 입력받음
                first_input = int(first_input)  # 0-26 인 숫자
                if 0 <= first_input <= 26:
                    if environment.board_r[first_input] != player:  # 그 위치에 자기 말이 없다면 wrong action
                        self.flag = 1
                    else:
                        location = int(input("What square would you like to move to? (0-26)"))  # 비어있는 곳으로 이동 가능하다.
                        action = str(first_input) + 'to' + str(location)
                else:
                    self.flag = 1

            else:  # 잘못된 입력
                self.flag = 1

            # 입력받은 행동이 가능한 행동이면 반복문을 탈출
            if action in available_action and self.flag == 0:
                if player == 1:
                    if size == 0:
                        environment.p1_small_piece -= 1
                    elif size == 1:
                        environment.p1_medium_piece -= 1
                    elif size == 2:
                        environment.p1_large_piece -= 1

                else:  # player == 2
                    if size == 0:
                        environment.p2_small_piece -= 1
                    elif size == 1:
                        environment.p2_medium_piece -= 1
                    elif size == 2:
                        environment.p2_large_piece -= 1
                # 옮기느라 사이즈가 없으면 그냥 return
                return action
            # 아니면 행동 입력을 반복
            else:
                print("You selected wrong action")
                environment.print_board()
                self.flag = 0
        return

# 랜덤 플레이어

In [4]:
class Random_player:
    def __init__(self):
        self.name = "Random player"

    def select_action(self, environment, player):
        # 가능한 행동 조사
        available_action = environment.get_action(player)
        # 가능한 행동 중 하나를 무작위로 선택
        action = np.random.randint(len(available_action))

        # 선택한 행동이 새로 말을 놓는 행동이라면 값을 빼주어야함.
        if isinstance(available_action[action], int):  # 숫자라면
            if player == 1:
                if 0 <= available_action[action] <= 8:
                    environment.p1_small_piece -= 1
                elif 9 <= available_action[action] <= 17:
                    environment.p1_medium_piece -= 1
                else:  # 18 <= action <= 26
                    environment.p1_large_piece -= 1
            else:  # player == 2:
                if 0 <= available_action[action] <= 8:
                    environment.p2_small_piece -= 1
                elif 9 <= available_action[action] <= 17:
                    environment.p2_medium_piece -= 1
                else:  # 18 <= action <= 26
                    environment.p2_large_piece -= 1

        return available_action[action]

# 몬테카를로 플레이어

In [5]:
class Monte_Carlo_player:
    def __init__(self):
        self.name = "MC player"
        self.num_playout = 300

    def select_action(self, environment, player):
        # 가능한 행동 조사
        available_action = environment.get_action(player)
        # 상태가치를 저장할 배열 V
        V = np.zeros(len(available_action))

        # 가능한 행동들을 돌면서 V[i]+=1을 해줌
        for i in range(len(available_action)):
            # 플레이 아웃을 1000번 반복
            for j in range(self.num_playout):
                # 지금 상태를 복사해서 플레이 아웃에 사용
                temp_env = copy.deepcopy(environment)
                # play out 의 결과는 승리플레이어의 값으로 반환
                # p1이 이기면 1, p2가 이기면 -1

                # i번째 행동이 새로 말을 놓는 행동이라면 값을 빼주어야함.
                if isinstance(available_action[i], int):  # 숫자라면
                    if 0 <= available_action[i] <= 8:
                        if player == 1:
                            temp_env.p1_small_piece -= 1
                        else:
                            temp_env.p2_small_piece -= 1
                    elif 9 <= available_action[i] <= 17:
                        if player == 1:
                            temp_env.p1_medium_piece -= 1
                        else:
                            temp_env.p2_medium_piece -= 1
                    else:  # 18 <= action <= 26
                        if player == 1:
                            temp_env.p1_large_piece -= 1
                        else:
                            temp_env.p2_large_piece -= 1

                self.playout(temp_env, available_action[i], player)
                if player == temp_env.reward:
                    V[i] += 1

        # 가장 승률이 높은 행동을 저장
        action = np.argmax(V)

        # 선택한 행동이 새로 말을 놓는 행동이라면 값을 빼주어야함.
        if isinstance(available_action[action], int):  # 숫자라면
            if 0 <= available_action[action] <= 8:
                if player == 1:
                    env.p1_small_piece -= 1
                else:
                    env.p2_small_piece -= 1
            elif 9 <= available_action[action] <= 17:
                if player == 1:
                    env.p1_medium_piece -= 1
                else:
                    env.p2_medium_piece -= 1
            else:  # 18 <= action <= 26
                if player == 1:
                    env.p1_large_piece -= 1
                else:
                    env.p2_large_piece -= 1

        return available_action[action]

    # 플레이 아웃 재귀함수
    # 게임이 종료상태 (승 또는 패 또는 비김)가 될때까지 행동을 임의로 선택하는 것을 반복
    # 플레이어는 계속 바뀌기 때문에 (-)를 곱해서 -1, 1, -1 이 되게 함.

    def playout(self, temp_env, action, player):
        # 보드에 플레이어의 선택을 표시(만약 새로운 말을 놓는 거라면)
        if isinstance(action, int):  # 숫자라면
            temp_env.board_r[action] = player
        else:  # 문자라면 -> 움직이는 액션이다
            action = action.split('to')  # pos[0]가 지울 장소, pos[1]이 생길 장소

            # 원래 있던 자리에 있는 말을 지우고, (real 보드에만 지우면 된다) 옮길 자리에 추가한다(마찬가지로 real 보드에만 추가하면 된다.)
            temp_env.board_r[int(action[0])] = 0
            temp_env.board_r[int(action[1])] = player

        temp_env.copy_real_to_vision()
        temp_env.end_check(player)
        # 게임 종료 체크
        if temp_env.done:
            # print("done")
            return
        else:
            # 플레이어 교체
            player = -player
            # 가능한 행동 조사
            available_action = temp_env.get_action(player)

            # 무작위로 행동을 선택
            action = np.random.randint(len(available_action))

            # 선택한 행동이 새로 말을 놓는 행동이라면 값을 빼주어야함.
            if isinstance(available_action[action], int):  # 숫자라면
                if 0 <= available_action[action] <= 8:
                    if player == 1:
                        temp_env.p1_small_piece -= 1
                    else:
                        temp_env.p2_small_piece -= 1
                elif 9 <= available_action[action] <= 17:
                    if player == 1:
                        temp_env.p1_medium_piece -= 1
                    else:
                        temp_env.p2_medium_piece -= 1
                else:  # 18 <= action <= 26
                    if player == 1:
                        temp_env.p1_large_piece -= 1
                    else:
                        temp_env.p2_large_piece -= 1

            self.playout(temp_env, available_action[action], player)

# Q-learning 플레이어

In [6]:
class Q_learning_player:
    def __init__(self):
        self.name = "Q_player"
        # Q-table을 딕셔너리로 정의
        self.qtable = {}
        # e-greedy 계수 정의
        self.epsilon = 1
        # 학습률 정의
        self.learning_rate = 0.1
        self.gamma = 0.9
        self.print = False

    # policy 에 따라 상태에 맞는 행동을 선택
    def select_action(self, environment, c_player):
        # policy 에 따라 행동을 선택
        action = self.policy(environment, c_player)
        return action

    def policy(self, environment, c_player):
        # 행동 가능한 상태를 저장
        available_action = environment.get_action(c_player)
        # 행동 가능한 상태의 Q-value를 저장
        qvalues = np.zeros(len(available_action))

        # 행동 가능한 상태의 Q-value를 조사
        for i, act in enumerate(available_action):
            key = (tuple(environment.board_r), act)

            # 현재 상태를 경험한 적이 없다면(딕셔너리에 없다면) 딕셔너리에 추가(Q-value = 0)
            if self.qtable.get(key) == None:
                self.qtable[key] = 0
            # 행동 가능한 상태의 Q-value 저장
            qvalues[i] = self.qtable.get(key)

        # e-greedy
        # 가능한 행동들 중에서 Q-value 가 가장 큰 행동을 저장
        greedy_action = np.argmax(qvalues)

        # max Q-value와 같은 값이 여러개 있는지 확인한 후 double_check에 상태를 저장
        double_check = (np.where(qvalues == np.max(qvalues), 1, 0))

        # 여러개 있다면 중복된 상태중에서 다시 무작위로 선택
        if np.sum(double_check) > 1:
            double_check = double_check / np.sum(double_check)
            greedy_action = np.random.choice(range(0, len(double_check)), p=double_check)
            
        # e-greedy 로 행동들의 선택 확률을 계산
        pr = np.zeros(len(available_action))

        for i in range(len(available_action)):
            if i == greedy_action:
                pr[i] = 1 - self.epsilon + self.epsilon / len(available_action)
                if pr[i] < 0:
                    print("{} : - pr".format(np.round(pr[i], 2)))
            else:
                pr[i] = self.epsilon / len(available_action)
                if pr[i] < 0:
                    print("{} : - pr".format(np.round(pr[i], 2)))

        action = np.random.choice(range(0, len(available_action)), p=pr)

        # 선택한 행동이 새로 말을 놓는 행동이라면 값을 빼주어야함.
        if isinstance(available_action[action], int):  # 숫자라면
            if 0 <= available_action[action] <= 8:
                if c_player == 1:
                    environment.p1_small_piece -= 1
                else:
                    environment.p2_small_piece -= 1
            elif 9 <= available_action[action] <= 17:
                if c_player == 1:
                    environment.p1_medium_piece -= 1
                else:
                    environment.p2_medium_piece -= 1
            else:  # 18 <= action <= 26
                if c_player == 1:
                    environment.p1_large_piece -= 1
                else:
                    environment.p2_large_piece -= 1

        action = available_action[action]
        return action
    
    def learn_qtable(self, board_backup, action_backup, environment, c_reward, c_player):
        # 현재 상태와 행동을 키로 저장
        key = (board_backup, action_backup)

        # Q-table 학습
        if environment.done:
            # 게임이 끝났을 경우 학습
            self.qtable[key] += self.learning_rate * (c_reward - self.qtable[key])
        else:
            # 게임이 진행중일 경우 학습
            # 다음 상태의 max Q 값 계산
            available_action = environment.get_action(c_player)

            qvalues = np.zeros(len(available_action))

            for i, act in enumerate(available_action):
                next_key = (tuple(env.board_r), act)

                # 다음 상태를 경험한 적이 없다면(딕셔너리에 없다면) 딕셔너리에 추가(Q-value = 0)
                if self.qtable.get(next_key) == None:
                    self.qtable[next_key] = 0
                qvalues[i] = self.qtable.get(next_key)

            # maxQ 조사
            maxQ = np.max(qvalues)

            # 게임이 진행중일 때 학습
            self.qtable[key] += self.learning_rate * (c_reward + self.gamma * maxQ - self.qtable[key])

# Q-learning 플레이어 훈련

In [7]:
p1_Qplayer = Q_learning_player()
p2_Qplayer = Q_learning_player()

# 입실론은 0.5로 설장
p1_Qplayer.epsilon = 0.5
p2_Qplayer.epsilon = 0.5

p1_score = 0
p2_score = 0
draw_score = 0
max_learn =100000

for j in tqdm(range(max_learn)):
    np.random.seed(j)
    env = Environment()
    for i in range(10000):
        # p1 행동 선택
        player = 1
        pos = p1_Qplayer.policy(env, player)
        
        # 현재 상태 s, 행동 a를 저장
        p1_board_backup = tuple(env.board_r)
        p1_action_backup = pos
        
        if isinstance(pos, int):  # 숫자라면
            env.board_r[pos] = player
        else:  # 문자라면 -> 움직이는 액션이다
            pos = pos.split('to')  # pos[0]가 지울 장소, pos[1]이 생길 장소
            env.board_r[int(pos[0])] = 0
            env.board_r[int(pos[1])] = player
        
        env.copy_real_to_vision()
        env.end_check(player)
        
        # 게임이 종료상태라면 각 플레이어의 Q-table을 학습
        if env.done:
            # 비겼으면 보수 0
            if env.reward == 0:
                p1_Qplayer.learn_qtable(p1_board_backup, p1_action_backup, env, 0, player)
                p2_Qplayer.learn_qtable(p2_board_backup, p2_action_backup, env, 0, player)
                draw_score += 1   
                break
            # p1이 이겼으므로 보상 +1
            # p2이 졌으므로 보상 -1
            else:
                p1_Qplayer.learn_qtable(p1_board_backup, p1_action_backup, env, 1, player)
                p2_Qplayer.learn_qtable(p2_board_backup, p2_action_backup, env, -1, player)
                p1_score += 1
                break

        # 게임이 끝나지 않았다면 p2의 Q-table을 학습 (게임 시작직후에는 p2는 학습할 수 없음)
        if i != 0:
            p2_Qplayer.learn_qtable(p2_board_backup, p2_action_backup, env, -0.01, player)
        
        # p2 행동 선택
        player = -1
        pos = p2_Qplayer.policy(env, player)
        
        p2_board_backup = tuple(env.board_r)
        p2_action_backup = pos
        
        if isinstance(pos, int):  # 숫자라면
            env.board_r[pos] = player
        else:  # 문자라면 -> 움직이는 액션이다
            pos = pos.split('to')  # pos[0]가 지울 장소, pos[1]이 생길 장소

            # 원래 있던 자리에 있는 말을 지우고, (real 보드에만 지우면 된다) 옮길 자리에 추가한다(마찬가지로 real 보드에만 추가하면 된다.)
            env.board_r[int(pos[0])] = 0
            env.board_r[int(pos[1])] = player
        
        env.copy_real_to_vision()
        env.end_check(player)

        if env.done:
            # 비겼으면 보수 0
            if env.reward == 0:
                p1_Qplayer.learn_qtable(p1_board_backup, p1_action_backup, env, 0, player)
                p2_Qplayer.learn_qtable(p2_board_backup, p2_action_backup, env, 0, player)
                draw_score += 1
                break
            # p2이 이겼으므로 보상 +1
            # p1이 졌으므로 보상 -1
            else:
                p1_Qplayer.learn_qtable(p1_board_backup, p1_action_backup, env, -1, player)
                p2_Qplayer.learn_qtable(p2_board_backup, p2_action_backup, env, 1, player)
                p2_score += 1
                break

        # 게임이 끝나지 않았다면 p1의 Q-table 학습
        p1_Qplayer.learn_qtable(p1_board_backup, p1_action_backup, env, -0.01, player)
        # # 1000 게임마다 게임 결과 표시
#         if j % 1000 == 0:
            # print("j = {} p1 = {} p2 = {} draw = {}".format(j, p1_score, p2_score, draw_score))
            
print("p1 = {} p2 = {} draw = {}".format(p1_score, p2_score, draw_score))
print("end train")

100%|█████████████████████████████████████████████████████████████████████████| 100000/100000 [13:30<00:00, 123.40it/s]

p1 = 56409 p2 = 42973 draw = 618
end train


In [11]:
# 인간 vs 인간
# p1 = Human_player()
# p2 = Human_player()

# 인간 vs 랜덤 플레이어
# p1 = Human_player()
# p2 = Random_player()

# 인간 vs MCTS 플레이어
# p1 = Human_player()
# p2 = Monte_Carlo_player()

# 인간 vs Q-learning Player
p1 = Human_player()
p2 = p2_Qplayer
p2.epsilon = 0

# 지정된 게임 수를 자동으로 두게 할 것인지 한 게임씩 두게 할 것인지 결정
# auto = True: 지정된 판 수(games)를 자동으로 진행
# auto = False: 한 게임씩 진행

auto = False
# auto = True

# auto 모드의 게임 수
games = 100
print("p1 player : {}".format(p1.name))
print("p2 player : {}".format(p2.name))

# 각 플레이어의 승리 횟수를 저장
p1_score = 0
p2_score = 0
draw_score = 0
if auto:
    
    # 자동 모드 실행
    for j in tqdm(range(games)):
        np.random.seed(j)
        env = Environment()
        for i in range(10000):
            
            # p1과 p2가 번갈아 가면서 게임을 진행
            # p1(1) -> p2(-1) -> p1(1) -> p2(-1) ...
            reward, done = env.move(p1, p2, (-1)**i)
            
            # 게임 종료 체크
            if done == True:
                if reward == 1:
                    print("j = {} winner is p1{}".format(j, p1.name))
                    p1_score += 1
                elif reward == -1:
                    print("j = {} winner is p2{}".format(j, p2.name))
                    p2_score += 1
                else:
                    print("j = {} draw".format(j))
                    draw_score += 1
                break
                
else:
    # 한 게임씩 진행하는 수동 모드
    while True:
        env = Environment()
        env.print = True
        for i in range(10000):
            reward, done = env.move(p1,p2,(-1)**i)
            
            # 게임 종료 체크
            if done == True:
                if reward == 1:
                    print("winner is p1({})".format(p1.name))
                    p1_score += 1
                elif reward == -1:
                    print("winner is p1({})".format(p2.name))
                    p2_score += 1
                else:
                    print("draw")
                    draw_score += 1
                break
        print("final result")
        env.print_board()
        
        # 최종 결과 출력
        print("final result")
        env.print_board()
        
        # 한 게임 더?
        answer = input("More Game? (y/n)")
        if(answer == "n"):
            break
print("p1({}) = {} p2({}) = {} draw = {}".format(p1.name, p1_score, p2.name, p2_score,draw_score))

p1 player : Human player
p2 player : Q_player
possible actions = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
+----+----+----+
+  0 +  1 +  2 +
+----+----+----+
+  3 +  4 +  5 +
+----+----+----+
+  6 +  7 +  8 +
+----+----+----+
0-26 to move, n to place a new piece?n
What size piece would you like to use? (0-2)2
What square would you like to move to? (0-8)4
+----+----+----+
|    |    |    |
+----+----+----+
|    | OL |    |
+----+----+----+
|    |    |    |
+----+----+----+
+----+----+----+
|    | XL |    |
+----+----+----+
|    | OL |    |
+----+----+----+
|    |    |    |
+----+----+----+
possible actions = [0, 2, 3, 5, 6, 7, 8, 9, 11, 12, 14, 15, 16, 17, 18, 20, 21, 23, 24, 25, 26, '22to18', '22to20', '22to21', '22to23', '22to24', '22to25', '22to26']
+----+----+----+
+  0 +  1 +  2 +
+----+----+----+
+  3 +  4 +  5 +
+----+----+----+
+  6 +  7 +  8 +
+----+----+----+
0-26 to move, n to place a new piece?n
What size piece would yo